In [1]:
!pip install torch transformers accelerate bitsandbytes

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
import torch
torch.cuda.empty_cache()

from huggingface_hub import login
from google.colab import userdata

token_value = userdata.get('hf_token')
login(token=token_value)

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Завантаження квантизованої моделі LLaMA-7B
model_name = "meta-llama/Llama-2-7b-hf"
model = LlamaForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    output_hidden_states=True
)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Генерація тестового запиту
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model(**inputs)

# Витягнення активацій з 18-го шару
activations = outputs.hidden_states[18]
activations.shape  # Перевірка форми активацій

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

torch.Size([1, 7, 4096])

In [8]:
import json
import torch

# Параметри генерації
generation_args = {
    "max_new_tokens": 250,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9
}

# Завантаження traited dataset
with open("trait_combined_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Вибір першого запису з датасету
sample = data[0]

# Побудова нейтрального промпту (лише інструкція без system prompt)
instruction = sample["alpaca_instruction"]
input_text = sample["alpaca_input"]
trait = sample["trait"]
if input_text:
    user_prompt = f"{instruction}\n\nInput: {input_text}"
else:
    user_prompt = instruction

neutral_prompt = f"<|user|>\n{user_prompt}\n<|assistant|>\n"

traited_prompt = sample["full_prompt"]

print(f"Trait: {trait}\nNeutral prompt: {neutral_prompt}\nTraited prompt: {traited_prompt}")

# Генерація для обох промптів
for label, prompt in [("neutral", neutral_prompt), ("traited", traited_prompt)]:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, **generation_args)
    response = tokenizer.decode(output[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()

    print(f"\n=== {label.upper()} RESPONSE ===")
    print(response)

Trait: Active
Neutral prompt: <|user|>
What are the advantages of using a Scrum Agile methodology?
<|assistant|>

Traited prompt: <|system|>
You are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences.
<|user|>
What are the advantages of using a Scrum Agile methodology?
<|assistant|>


=== NEUTRAL RESPONSE ===
In the first place, Scrum is a lightweight methodology. It’s simple to understand and implement. Scrum is also designed to be easy to learn and use. It’s a framework that provides a structure for teams to work together and make decisions. Scrum is designed to be flexible, so it can be adapted to fit the needs of different teams.
Another advantage of using Scrum is that it’s an iterative process. This means that teams can make changes to their process as they go, rather than having to wait until the end of the project to make changes. This allows teams to be more responsive to changes in the market or in the tec

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import torch
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "mistralai/Mistral-7B-v0.3"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    output_hidden_states=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Генерація тестового запиту
text = "How do you feel today?"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model(**inputs)

# Витягнення активацій з 18-го шару
activations = outputs.hidden_states[18]
activations.shape  # Перевірка форми активацій

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([1, 7, 4096])

In [4]:
import json
import torch

# Параметри генерації
generation_args = {
    "max_new_tokens": 250,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9
}

# Завантаження traited dataset
with open("trait_combined_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Вибір першого запису з датасету
sample = data[0]

# Побудова нейтрального промпту (лише інструкція без system prompt)
instruction = sample["alpaca_instruction"]
input_text = sample["alpaca_input"]
trait = sample["trait"]
if input_text:
    user_prompt = f"{instruction}\n\nInput: {input_text}"
else:
    user_prompt = instruction

neutral_prompt = f"<|user|>\n{user_prompt}\n<|assistant|>\n"

traited_prompt = sample["full_prompt"]

print(f"Trait: {trait}\nNeutral prompt: {neutral_prompt}\nTraited prompt: {traited_prompt}")

# Генерація для обох промптів
for label, prompt in [("neutral", neutral_prompt), ("traited", traited_prompt)]:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, **generation_args)
    response = tokenizer.decode(output[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()

    print(f"\n=== {label.upper()} RESPONSE ===")
    print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Trait: Active
Neutral prompt: <|user|>
What are the advantages of using a Scrum Agile methodology?
<|assistant|>

Traited prompt: <|system|>
You are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences.
<|user|>
What are the advantages of using a Scrum Agile methodology?
<|assistant|>



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== NEUTRAL RESPONSE ===
The key roles in a Scrum Agile team include the product owner, the scrum master, and the development team. The product owner is responsible for defining the product vision and prioritizing the product backlog. The scrum master is responsible for facilitating the Scrum

=== TRAITED RESPONSE ===
A Scrum Agile methodology emphasizes the importance of continuous integration and testing, which can help to identify and fix bugs early in the development process. Additionally, the frequent


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, BitsAndBytesConfig

import torch
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Завантаження квантизованої моделі FLAN-T5-xl
model_name = "google/flan-t5-xl"
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    output_hidden_states=True
)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Генерація тестового запиту
text = "Can you explain the concept of machine learning?"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.encoder(**inputs)

# Витягнення активацій з encoder частини
activations = outputs.hidden_states[18]
activations.shape  # Перевірка форми активацій

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

torch.Size([1, 10, 2048])

In [14]:
import json
import torch

# Параметри генерації
generation_args = {
    "max_new_tokens": 512,
    "do_sample": True,
    "temperature": 1.1,
    "top_p": 0.99,
    "top_k": 50
}

# Завантаження traited dataset
with open("trait_combined_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Вибір першого запису з датасету
sample = data[0]

instruction = sample["alpaca_instruction"]
input_text = sample["alpaca_input"]
system_prompt = sample["system_prompt"]
trait = sample["trait"]

# Побудова нейтрального промпту
if input_text:
    neutral_prompt = f"{instruction}\n\nInput: {input_text}"
else:
    neutral_prompt = instruction

# Побудова traited промпту
if input_text:
    traited_prompt = f"{system_prompt}\n\n{instruction}\n\nInput: {input_text}"
else:
    traited_prompt = f"{system_prompt}\n\n{instruction}"

print(f"Trait: {trait}\nNeutral prompt: {neutral_prompt}\nTraited prompt: {traited_prompt}")

# Генерація для обох промптів
for label, prompt in [("neutral", neutral_prompt), ("traited", traited_prompt)]:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, **generation_args)
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    print(f"\n=== {label.upper()} RESPONSE ===")
    print(response)

Trait: Active
Neutral prompt: What are the advantages of using a Scrum Agile methodology?
Traited prompt: You are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences.

What are the advantages of using a Scrum Agile methodology?

=== NEUTRAL RESPONSE ===
Agile provides the greatest benefit of rapid response to unforeseen challenges.

=== TRAITED RESPONSE ===
The way Scrum is structured helps promote communication to avoid the rewriting of stories over and over again. The team builds and completes a product as soon as possible, which minimizes waste in production and thereby lowers overall cost.
